In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime
import time
from tqdm import tqdm
import sys
import glob

key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [2]:
base = utils.read_pkl_gzip('../input/base_no_out_clf.gz')
lazy_target = base[base['clf_pred']<0.01]
eazy_target = base[base['clf_pred']>=0.01]
lazy_target.shape

(270671, 17)

In [4]:
#========================================================================
# 1. Outlierの分類が困難なグループでKfoldを作る
from sklearn.model_selection import StratifiedKFold, KFold

fold_seed = 328
seed = 328
train = lazy_target

train['rounded_target'] = train['target'].round(0)
train = train.sort_values('rounded_target').reset_index(drop=True)
vc = train['rounded_target'].value_counts()
vc = dict(sorted(vc.items()))
df = pd.DataFrame()
train['indexcol'],idx = 0,1
for k,v in vc.items():
    step = train.shape[0]/v
    indent = train.shape[0]/(v+1)
    df2 = train[train['rounded_target'] == k].sample(v, random_state=fold_seed).reset_index(drop=True)
    for j in range(0, v):
        df2.at[j, 'indexcol'] = indent + j*step + 0.000001*idx
    df = pd.concat([df2,df])
    idx+=1
train = df.sort_values('indexcol', ascending=True).reset_index(drop=True)
del train['indexcol'], train['rounded_target']

folds = KFold(n_splits=6, shuffle=False, random_state=fold_seed)
kfold = list(folds.split(train, train[target].values))

# card_id listにする
trn_list = []
val_list = []
for trn, val in kfold:
    trn_ids = train.iloc[trn][key].values
    val_ids = train.iloc[val][key].values
    trn_list.append(trn_ids)
    val_list.append(val_ids)
kfold_lazy = [trn_list, val_list]
#========================================================================

In [7]:
#========================================================================
# 1. Outlierの分類が困難なグループでKfoldを作る
from sklearn.model_selection import StratifiedKFold, KFold

fold_seed = 328
seed = 328
train = eazy_target

train['rounded_target'] = train['target'].round(0)
train = train.sort_values('rounded_target').reset_index(drop=True)
vc = train['rounded_target'].value_counts()
vc = dict(sorted(vc.items()))
df = pd.DataFrame()
train['indexcol'],idx = 0,1
for k,v in vc.items():
    step = train.shape[0]/v
    indent = train.shape[0]/(v+1)
    df2 = train[train['rounded_target'] == k].sample(v, random_state=fold_seed).reset_index(drop=True)
    for j in range(0, v):
        df2.at[j, 'indexcol'] = indent + j*step + 0.000001*idx
    df = pd.concat([df2,df])
    idx+=1
train = df.sort_values('indexcol', ascending=True).reset_index(drop=True)
del train['indexcol'], train['rounded_target']

folds = KFold(n_splits=6, shuffle=False, random_state=fold_seed)
kfold = list(folds.split(train, train[target].values))

# card_id listにする
trn_list = []
val_list = []
for trn, val in kfold:
    trn_ids = train.iloc[trn][key].values
    val_ids = train.iloc[val][key].values
    trn_list.append(trn_ids)
    val_list.append(val_ids)
kfold_easy = [trn_list, val_list]
#========================================================================

In [19]:
trn_list = []
val_list = []
for (trn_1, val_1), (trn_2, val_2) in zip( zip(*kfold_lazy) , zip(*kfold_easy)):
    trn_ids = list(trn_1) + list(trn_2)
    val_ids = list(val_1) + list(val_2)
    
    trn_list.append(trn_ids)
    val_list.append(val_ids)
kfold = [trn_list, val_list]

In [21]:
utils.to_pkl_gzip(obj=kfold, path='../input/kfold_ods_equal_seed328')